In [ ]:
import os
os.environ["XDG_CACHE_HOME"] = "/home/olab/tomerronen1/xdg_cache"
from datasets import load_dataset

dataset = load_dataset("hotpot_qa", "distractor")
dataset


In [ ]:
dataset["validation"]

In [ ]:
dataset["validation"][0]["context"]

In [ ]:
dataset["validation"][1]["supporting_facts"]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("ck46/t5-base-hotpot-qa-qg")
model = AutoModelForSeq2SeqLM.from_pretrained("ck46/t5-base-hotpot-qa-qg")


In [ ]:
qasper = load_dataset("tau/sled", "hotpotqa")

In [ ]:
qasper["validation"]

In [ ]:
def prepare_example(example):
    context = []
    for title, sentences in zip(example["context"]["title"], example["context"]["sentences"]):
        context.append(title)
        context.append(":\n")
        context.extend(sentences)
        context.append('\n\n')
    context.pop()
    context = ''.join(context) 

    question = f"Question:\n{example['question']}\n"
    prepared_example = {"id": example["id"], "pid": example["id"] + "_0", "input": context,
                        "input_prefix": question, "output": example["answer"]}
    return prepared_example


# dataset["validation"].select(range(10)).map(prepare_example)[0]
prepared_dataset_dict = dataset.map(prepare_example)


In [ ]:
prepared_dataset_dict.push_to_hub("tomer/sled_hotpotqa_distractor", private=True)


In [ ]:
raise

In [ ]:
example = dataset["train"][6]

context = []
for title, sentences in zip(example["context"]["title"], example["context"]["sentences"]):
    context.append(title)
    context.append(":\n")
    context.extend(sentences)
    context.append('\n\n')
context.pop()
context = ''.join(context) 

question = f"Question:\n{example['question']}\n"

print(question + '\n' + context)

# print(context)
# print(question)
# print(text)

In [ ]:
batch = tokenizer(text, return_tensors="pt")
max_length = tokenizer.model_max_length
batch["input_ids"] = batch["input_ids"][:,-max_length:]
batch["attention_mask"] = batch["attention_mask"][:,-max_length:]
prediction = tokenizer.batch_decode(model.generate(**batch, num_beams=1, max_length=20))[0]
print(prediction)
print(example["answer"])